In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [74]:
data = pd.read_csv("Project Information for Report.csv")

#data.drop(columns=["Unnamed: 7"], inplace=True)
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
for col in data.columns:
    data[col] = data[col].astype(str).str.replace(',', '.', regex=False)

numeric_columns = ['engine_hours', 'CO2', 'CO2/h', 'Idle %', 'Fuel (l)']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [70]:
data.to_csv("Project Information for Report.csv", index=False)
data.dropna()
data

,Machine,engine_hours,Date,CO2,CO2/h,Idle %,Fuel (l)
0,P40.17,0.5667,2025-04-14,6.7,11.7,20.6,2.5
1,P40.17,0.6833,2025-04-15,6.1,9.0,56.1,2.3
2,P40.17,0.5833,2025-04-16,6.7,11.4,9.2,2.5
3,P40.17,1.7667,2025-04-22,15.2,8.6,27.4,5.7
4,P40.17,1.8500,2025-04-23,12.2,6.6,7.5,4.6
...,...,...,...,...,...,...,...
151,250EL,0.5000,2025-05-08,14.1,28.2,11.4,5.3
152,250EL,0.3167,2025-05-09,3.7,11.8,27.7,1.4
153,250EL,2.9000,2025-05-12,57.5,19.8,48.2,21.6
154,250EL,4.0500,2025-05-13,91.2,22.5,21.5,34.3


### Daily CO₂ emissions by machine


In [76]:
grouped_co2 = data.groupby(['Date', 'Machine'])['CO2'].sum().reset_index()
fig1 = px.bar(grouped_co2, x='Date', y='CO2', color='Machine', title='Daily CO₂ Emissions by Machine', barmode='stack')
fig1.show()

### Idle % over time by Machine

In [77]:
fig2 = px.line(data, x='Date', y='Idle %', color='Machine', title='Idle % Over Time')
fig2.show()

### CO2/h vs Idle %

In [55]:
fig3 = px.scatter(data, x='Idle %', y='CO2/h', color='Machine',
                  hover_data=['Date', 'engine_hours', 'Fuel (l)'],
                  title='CO₂ per Hour vs Idle %')
fig3.show()

### 4. Engine Hours and Fuel by machine


In [56]:
for machine in data['Machine'].unique():
    filtered = data[data['Machine'] == machine]
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(x=filtered['Date'], y=filtered['engine_hours'], mode='lines+markers', name='Engine Hours'))
    fig4.add_trace(go.Scatter(x=filtered['Date'], y=filtered['Fuel (l)'], mode='lines+markers', name='Fuel (l)', yaxis='y2'))
    fig4.update_layout(title=f'Engine Hours and Fuel: ({machine})',
                       yaxis=dict(title='Engine Hours'),
                       yaxis2=dict(title='Fuel (l)', overlaying='y', side='right'))
    fig4.show()


### Total CO₂ and Fuel by Machine

In [78]:
aggr = data.groupby('Machine')[['CO2', 'Fuel (l)']].sum().reset_index()
fig5a = px.bar(aggr, x='Machine', y='CO2', title='Total CO₂ by Machine', color='CO2')
fig5a.show()

In [79]:
fig5b = px.bar(aggr, x='Machine', y='Fuel (l)', title='Total Fuel Used by Machine', color="Fuel (l)",
              color_continuous_scale=px.colors.sequential.Viridis)
fig5b.show()

# 6. Idle % Heatmap (Date x Machine)

In [80]:
pivot = data.pivot_table(index='Machine', columns='Date', values='Idle %')
fig6 = px.imshow(pivot, aspect='auto', color_continuous_scale='Viridis',
                 labels={'color': 'Idle %'}, title='Idle % Heatmap')
fig6.show()

### Cumulative CO₂ Emissions Over Time


In [81]:
df_sorted = data.sort_values('Date')
df_sorted['Cumulative CO2'] = df_sorted.groupby('Machine')['CO2'].cumsum()
fig7 = px.line(df_sorted, x='Date', y='Cumulative CO2', color='Machine', title='Cumulative CO₂ Over Time')
fig7.show()